## Analyzing Optics Camera Coverage Report

[CameraCoverageReport - S3](https://s3.console.aws.amazon.com/s3/buckets/dcss-optics-camera-onboard-progress?region=us-east-1&tab=objects)

In [1]:
import re
import pandas as pd
from pandas import DataFrame as Data
from IPython.display import display, Markdown, HTML
from Data.NTE_Sites_Data import NTE_Sites, NTE_BMN_sites, ACE_EPOP_Sites, NTE_EPOP

In [2]:
new_csv = input("Enter the path to the csv file: ")
data = pd.read_csv(new_csv, low_memory=False)

In [3]:
data["Lenel:Name"] = data["Lenel:Name"].fillna("")
data.loc[
    data["Lenel:Name"].str.startswith("uc_"), "Post Deployment Status"
] = "UNDER_CONSTRUCTION"
data["Lenel:Name"] = data["Lenel:Name"].apply(
    lambda x: x.replace("uc_", "") if x.startswith("uc_") else x
)

**Cameras in Lenel vs Optics**

In [4]:
O = Data(data, columns=['In Optics'])
L = Data(data, columns=['In Lenel'])
non_exclusive = data[(data['Network'] != 'DSN') & (data['In Optics'] == 'Y')]
camInOptics = len(non_exclusive)
LenNoOptics = (O == 'N')

**NTE Cameras**

In [5]:
data["Site Code"] = data["Lenel:Name"].str.extract(r'([A-Z]{3}\d{1,3})', expand=False)
data["NTE"] = data["Site Code"].isin(NTE_EPOP)
Lenel = data.loc[(data["In Optics"] == "N") & (data["Lenel:Name"].notnull())]
LenelnoE = Lenel[~Lenel['Site Code'].isin(NTE_EPOP)]
Lenel_UC = Lenel.loc[Lenel["Post Deployment Status"] == "UNDER_CONSTRUCTION", "Lenel:Name"].count()
NTEtot = data[data['Site Code'].isin(NTE_EPOP)]
NTE_UC = NTEtot[NTEtot['Post Deployment Status'] == 'UNDER_CONSTRUCTION']

In [6]:
display(Markdown('# Optics Report Summary'))
display(Markdown(f'**Total cameras in Optics (DSN excluded): <span style="color:green">{camInOptics}**</span>'))
display(Markdown(f'**Cameras currently in Lenel, but not Optics: <span style="color:green">{LenNoOptics.sum().item()}**</span>'))
display(Markdown(f'**Total non-NTE Cameras in Lenel, but not Optics: <span style="color:green">{LenelnoE["Lenel:Name"].count()}**</span>'))
display(Markdown(f'**Number of non-NTE cameras in "UC" status: <span style="color:green">{Lenel_UC}**</span>'))
display(Markdown(f'**Total NTE Cameras in Lenel, but not Optics: <span style="color:green">{NTEtot["Lenel:Name"].count()}</span>**'))
display(Markdown(f'**Number of NTE Cameras in "UC" status: <span style="color:green">{NTE_UC.shape[0]}**</span>'))

# Optics Report Summary

**Total cameras in Optics (DSN excluded): <span style="color:green">97077**</span>

**Cameras currently in Lenel, but not Optics: <span style="color:green">6843**</span>

**Total non-NTE Cameras in Lenel, but not Optics: <span style="color:green">3262**</span>

**Number of non-NTE cameras in "UC" status: <span style="color:green">1802**</span>

**Total NTE Cameras in Lenel, but not Optics: <span style="color:green">4907</span>**

**Number of NTE Cameras in "UC" status: <span style="color:green">95**</span>